In [1]:
import pandas as pd 
import os 
pd.set_option('display.max_columns', None)  # Tüm sütunları gösterir
pd.set_option('display.max_colwidth', None) # Sütun genişliklerini sınırlandırmadan gösterir

In [ ]:
#fbrefscrape ile scrape edilen maç linkleri df inden istediğimiz linkleri ayıklayıp filtered_week_match_links.txt olarak scrape e gönderir.
df = pd.read_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv', parse_dates=['date'])
week_match_links = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_match_links.json')  
all_team_names = list(set(list(df.home_team.unique()) + list(df.away_team.unique()))) # -> datada bulunan tum takimlar. linklerden istediklerimizi çekmek için lazım olcak.
eklenmesi_gereken_takimlar = ['fenerbahce','adana-demirspor','besiktas','eyupspor'] #-> link kısmı turkce karakter icermiyor df iceriyor
all_team_names = all_team_names + eklenmesi_gereken_takimlar 
filtered_week_match_links = week_match_links[week_match_links['url'].str.contains('|'.join(all_team_names), case=False)]
filtered_week_match_links = list(filtered_week_match_links.url.values)
directory = '/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/sofa_temiz_urls'
file_path = os.path.join(directory, f'filtered_week_match_links.txt')
with open(file_path, 'w') as file:
    for url in filtered_week_match_links:
        file.write(url + '\n')

In [2]:
# BURAYI BASMADAN ONCE GEREKLI VERILERDE EKSIK VAR MI KONTROL ET DUZELT (EKSIK SARI VE KIRMIZI KART SORUN DEGIL)
df = pd.read_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv', parse_dates=['date'])
week_matches = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_matches.json',convert_dates= False)
week_matches_first_half = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_matches_first_half.json')
merged_week_matches = pd.merge(week_matches,week_matches_first_half, on= 'match_link', how='left')
merged_week_matches = merged_week_matches[merged_week_matches.Organizasyon.isin(df.organizasyon.unique())] #-> istediğim organizsayon isimlerni çeker.
merged_week_matches = merged_week_matches[merged_week_matches.ulke.isin(df.ulke.unique())] #-> ulkeler icin tekrar bi ayikladim bazen bir organizasyon iki ulkede olabiliyor.
# kart verileri eksikse 0 oldugundan eksik oluyorlar onları 0 yaptım 
merged_week_matches['Yellow cards_home'] = merged_week_matches['Yellow cards_home'].fillna(0).astype(int)
merged_week_matches['Yellow cards_away'] = merged_week_matches['Yellow cards_away'].fillna(0).astype(int)
merged_week_matches['Red cards_home'] = merged_week_matches['Red cards_home'].fillna(0).astype(int)
merged_week_matches['Red cards_away'] = merged_week_matches['Red cards_away'].fillna(0).astype(int)
merged_week_matches['Yellow cards_home_firsthalf'] = merged_week_matches['Yellow cards_home_firsthalf'].fillna(0).astype(int)
merged_week_matches['Yellow cards_away_firsthalf'] = merged_week_matches['Yellow cards_away_firsthalf'].fillna(0).astype(int)
merged_week_matches['Red cards_home_firsthalf'] = merged_week_matches['Red cards_home_firsthalf'].fillna(0).astype(int)
merged_week_matches['Red cards_away_firsthalf'] = merged_week_matches['Red cards_away_firsthalf'].fillna(0).astype(int)
merged_week_matches['date'] = pd.to_datetime(merged_week_matches['date'], format='%d-%m-%Y') #--> date todatetime a cevrildi
merged_week_matches.columns = merged_week_matches.columns.str.lower().str.replace(' ', '_') #--> sutun isimleri kucuk harf oldu ve bosluklar alt tire ile degisti.
# season_round ve season_period sutunlarını olusturur.
df_leagues = merged_week_matches[~(merged_week_matches.ulke == 'Europe')]
df_leagues.loc[ : , 'season_round'] = df_leagues.org_detay.apply(lambda x: x[-2:]).astype(int)
df_leagues = df_leagues[['match_link','season_round']]
df_leagues.loc[df_leagues.season_round <= 9 , 'season_period'] = 1
df_leagues.loc[(df_leagues.season_round > 9) & (df_leagues.season_round <= 18)  , 'season_period'] = 2
df_leagues.loc[(df_leagues.season_round > 18) & (df_leagues.season_round <= 27)  , 'season_period'] = 3
df_leagues.loc[(df_leagues.season_round >= 28)  , 'season_period'] = 4
merged_week_matches = pd.merge(merged_week_matches,df_leagues, on= 'match_link', how= 'left')
merged_week_matches = merged_week_matches.drop_duplicates()
merged_week_matches = merged_week_matches.fillna(0)
merged_week_matches.season_period = merged_week_matches.season_period.astype(int)
merged_week_matches.season_round = merged_week_matches.season_round.astype(int)
merged_week_matches.reset_index(drop= True, inplace= True)
# ikincyi yari sutunları olusturur.
merged_week_matches['total_shots_away_secondhalf'] = merged_week_matches['total_shots_away'] - merged_week_matches['total_shots_away_firsthalf']
merged_week_matches['total_shots_home_secondhalf'] = merged_week_matches['total_shots_home'] - merged_week_matches['total_shots_home_firsthalf']
merged_week_matches['shots_on_target_home_secondhalf'] = merged_week_matches['shots_on_target_home'] - merged_week_matches['shots_on_target_home_firsthalf']
merged_week_matches['shots_on_target_away_secondhalf'] = merged_week_matches['shots_on_target_away'] - merged_week_matches['shots_on_target_away_firsthalf']
merged_week_matches['shots_inside_box_home_secondhalf'] = merged_week_matches['shots_inside_box_home'] - merged_week_matches['shots_inside_box_home_firsthalf']
merged_week_matches['shots_inside_box_away_secondhalf'] = merged_week_matches['shots_inside_box_away'] - merged_week_matches['shots_inside_box_away_firsthalf']
merged_week_matches['shots_outside_box_home_secondhalf'] = merged_week_matches['shots_outside_box_home'] - merged_week_matches['shots_outside_box_home_firsthalf']
merged_week_matches['shots_outside_box_away_secondhalf'] = merged_week_matches['shots_outside_box_away'] - merged_week_matches['shots_outside_box_away_firsthalf']
merged_week_matches['corner_kicks_home_secondhalf'] = merged_week_matches['corner_kicks_home'] - merged_week_matches['corner_kicks_home_firsthalf']
merged_week_matches['corner_kicks_away_secondhalf'] = merged_week_matches['corner_kicks_away'] - merged_week_matches['corner_kicks_away_firsthalf']
merged_week_matches['passes_home_secondhalf'] = merged_week_matches['passes_home'] - merged_week_matches['passes_home_firsthalf']
merged_week_matches['passes_away_secondhalf'] = merged_week_matches['passes_away'] - merged_week_matches['passes_away_firsthalf']
merged_week_matches['yellow_cards_home_secondhalf'] = merged_week_matches['yellow_cards_home'] - merged_week_matches['yellow_cards_home_firsthalf']
merged_week_matches['yellow_cards_away_secondhalf'] = merged_week_matches['yellow_cards_away'] - merged_week_matches['yellow_cards_away_firsthalf']
merged_week_matches['red_cards_home_secondhalf'] = merged_week_matches['red_cards_home'] - merged_week_matches['red_cards_home_firsthalf']
merged_week_matches['red_cards_away_secondhalf'] = merged_week_matches['red_cards_away'] - merged_week_matches['red_cards_away_firsthalf']
merged_week_matches['free_kicks_home_secondhalf'] = merged_week_matches['free_kicks_home'] - merged_week_matches['free_kicks_home_firsthalf']
merged_week_matches['free_kicks_away_secondhalf'] = merged_week_matches['free_kicks_away'] - merged_week_matches['free_kicks_away_firsthalf']
#--> 2024-2025 sezonu season 8 olarak alıncak
merged_week_matches['season'] = 8 
# result ve iy_result sütunaları 
merged_week_matches['result'] = merged_week_matches.apply(lambda row: 1 if row['home_goal'] > row['away_goal'] 
                        else 2 if row['home_goal'] < row['away_goal'] 
                        else 0, axis=1)
merged_week_matches['iy_result'] = merged_week_matches.apply(lambda row: 1 if row['home_iy_goal'] > row['away_iy_goal'] 
                        else 2 if row['home_goal'] < row['away_goal'] 
                        else 0, axis=1)
# istedigim sutunları aldım sadece 
merged_week_matches = merged_week_matches[['date', 'hour', 'ulke', 'organizasyon', 'org_detay', 'home_team','home_goal', 'away_goal', 'away_team', 'result', 'home_iy_goal','away_iy_goal', 
'iy_result', 'corner_kicks_home','corner_kicks_home_firsthalf', 'corner_kicks_away','corner_kicks_away_firsthalf', 'total_shots_home','total_shots_home_firsthalf', 'shots_on_target_home',
'shots_on_target_home_firsthalf', 'total_shots_away','total_shots_away_firsthalf', 'shots_on_target_away','shots_on_target_away_firsthalf', 'fouls_home', 'fouls_away','yellow_cards_home', 
'yellow_cards_home_firsthalf', 'yellow_cards_away','yellow_cards_away_firsthalf', 'red_cards_home','red_cards_home_firsthalf', 'red_cards_away','red_cards_away_firsthalf', 'referee', 'red_card_avg',
'yellow_card_avg', 'shots_inside_box_home', 'shots_inside_box_away','shots_outside_box_home', 'shots_outside_box_away', 'free_kicks_home','free_kicks_away', 'ball_possession_home', 'ball_possession_away',
'passes_home', 'passes_away', 'ball_possession_home_firsthalf','ball_possession_away_firsthalf', 'passes_home_firsthalf','passes_away_firsthalf', 'free_kicks_home_firsthalf','free_kicks_away_firsthalf',
'shots_inside_box_home_firsthalf','shots_inside_box_away_firsthalf', 'shots_outside_box_home_firsthalf','shots_outside_box_away_firsthalf', 'total_shots_home_secondhalf','total_shots_away_secondhalf', 
'shots_on_target_home_secondhalf','shots_on_target_away_secondhalf', 'shots_inside_box_home_secondhalf','shots_inside_box_away_secondhalf', 'shots_outside_box_home_secondhalf','corner_kicks_home_secondhalf', 
'corner_kicks_away_secondhalf','passes_home_secondhalf', 'passes_away_secondhalf','free_kicks_home_secondhalf', 'yellow_cards_home_secondhalf','yellow_cards_away_secondhalf', 'red_cards_home_secondhalf',
'red_cards_away_secondhalf', 'free_kicks_away_secondhalf', 'match_link','season', 'season_round', 'season_period']]

In [3]:
merged_week_matches.dtypes

date                          datetime64[ns]
hour                                  object
ulke                                  object
organizasyon                          object
org_detay                             object
                                   ...      
free_kicks_away_secondhalf             int64
match_link                            object
season                                 int64
season_round                           int64
season_period                          int64
Length: 79, dtype: object

In [4]:
df_concanated = pd.concat([df,merged_week_matches])

In [5]:
df_concanated.sort_values(by= 'date', ascending= False).reset_index(drop= True, inplace= True)

In [7]:
df_concanated.sort_values(by= 'date', ascending= False)

,date,hour,ulke,organizasyon,org_detay,home_team,home_goal,away_goal,away_team,result,home_iy_goal,away_iy_goal,iy_result,corner_kicks_home,corner_kicks_home_firsthalf,corner_kicks_away,corner_kicks_away_firsthalf,total_shots_home,total_shots_home_firsthalf,shots_on_target_home,shots_on_target_home_firsthalf,total_shots_away,total_shots_away_firsthalf,shots_on_target_away,shots_on_target_away_firsthalf,fouls_home,fouls_away,yellow_cards_home,yellow_cards_home_firsthalf,yellow_cards_away,yellow_cards_away_firsthalf,red_cards_home,red_cards_home_firsthalf,red_cards_away,red_cards_away_firsthalf,referee,red_card_avg,yellow_card_avg,shots_inside_box_home,shots_inside_box_away,shots_outside_box_home,shots_outside_box_away,free_kicks_home,free_kicks_away,ball_possession_home,ball_possession_away,passes_home,passes_away,ball_possession_home_firsthalf,ball_possession_away_firsthalf,passes_home_firsthalf,passes_away_firsthalf,free_kicks_home_firsthalf,free_kicks_away_firsthalf,shots_inside_box_home_firsthalf,shots_inside_box_away_firsthalf,shots_outside_box_home_firsthalf,shots_outside_box_away_firsthalf,total_shots_home_secondhalf,total_shots_away_secondhalf,shots_on_target_home_secondhalf,shots_on_target_away_secondhalf,shots_inside_box_home_secondhalf,shots_inside_box_away_secondhalf,shots_outside_box_home_secondhalf,corner_kicks_home_secondhalf,corner_kicks_away_secondhalf,passes_home_secondhalf,passes_away_secondhalf,free_kicks_home_secondhalf,yellow_cards_home_secondhalf,yellow_cards_away_secondhalf,red_cards_home_secondhalf,red_cards_away_secondhalf,free_kicks_away_secondhalf,match_link,season,season_round,season_period
1,2024-08-12,21:00,Türkiye,Super Lig,Round 1,Rizespor,1,1,Başakşehir,0,1,1,0,12,8,1,0,23,13,5,3,6,1,1,1,14,14,0,0,2,0,0,0,0,0,Direnc Tonusluoglu,0.16,3.97,16,3,7,3,14,13,59%,41%,423,304,71%,29%,228,99,7,8,11,1,2,0,10,5,2,0,5,2,5,4,1,195,205,7,0,2,0,0,5,https://www.sofascore.com/football/match/basaksehir-fk-caykur-rizespor/olbsLlb#id:12528187,8,1,1
2,2024-08-12,22:15,Portugal,Liga Portugal Betclic,Round 1,Arouca,0,1,Vitória,2,0,1,2,6,2,5,3,8,5,0,0,10,6,4,3,13,11,7,2,2,0,1,0,0,0,Gustavo Correia,0.50,5.51,2,8,6,2,8,13,64%,36%,551,308,63%,37%,296,174,3,7,1,6,4,0,3,4,0,1,1,2,2,4,2,255,134,5,5,2,1,0,6,https://www.sofascore.com/football/match/arouca-vitoria-sc/jkbsCpk#id:12512898,8,1,1
0,2024-08-12,21:00,Türkiye,Super Lig,Round 1,Bodrum,0,1,Gaziantep,2,0,0,2,4,2,1,0,16,8,3,2,5,3,1,0,13,9,2,1,2,0,0,0,1,0,Mehmet Turkmen,0.29,4.16,9,3,7,2,8,13,62%,38%,444,276,47%,53%,172,196,4,9,7,1,1,2,8,2,1,1,2,2,6,2,1,272,80,4,1,2,0,1,4,https://www.sofascore.com/football/match/bodrum-gaziantep-fk/NccsPXFb#id:12528193,8,1,1
19,2024-08-11,15:30,Netherlands,Eredivisie,Round 1,Utrecht,1,0,Zwolle,1,1,0,1,2,1,1,0,9,8,2,2,5,0,1,0,10,18,0,0,3,1,0,0,0,0,Jannick van der Laan,0.12,2.96,8,4,1,1,17,10,50%,50%,450,460,55%,45%,260,211,11,4,7,0,1,0,1,5,0,1,1,4,0,1,1,190,249,6,0,2,0,0,6,https://www.sofascore.com/football/match/pec-zwolle-fc-utrecht/Yibswjb#id:12449874,8,1,1
15,2024-08-11,19:15,Türkiye,Super Lig,Round 1,Sivasspor,0,0,Trabzonspor,0,0,0,0,5,2,7,4,14,5,5,2,16,6,2,1,12,11,1,1,0,0,0,0,0,0,Cihan Aydin,0.14,3.98,11,10,3,6,11,12,32%,68%,233,522,32%,68%,112,256,6,7,4,3,1,3,9,10,3,1,7,7,2,3,3,121,266,5,0,0,0,0,5,https://www.sofascore.com/football/match/sivasspor-trabzonspor/blbsBlb#id:12528185,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080,2017-08-05,21,France,Ligue 1,Round 1,Saint-Étienne,1,0,Nice,1,1,0,1,2,2,7,2,8,6,3,2,6,3,2,2,11,8,2,1,1,0,0,0,0,0,François Letexier,0.28,3.96,4,4,4,2,8,11,40,60,448,653,41,59,262,369,2,5,3,2,3,1,2,3,1,0,1,2,1,0,5,186,284,6,1,1,0,0,6,https://www.sofascore.com/football/match/saint-etienne-nice/lIsDI#id:7438247,1,1,1
20081,2017-08-05,21

In [9]:
merged_week_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 79 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   date                               27 non-null     datetime64[ns]
 1   hour                               27 non-null     object        
 2   ulke                               27 non-null     object        
 3   organizasyon                       27 non-null     object        
 4   org_detay                          27 non-null     object        
 5   home_team                          27 non-null     object        
 6   home_goal                          27 non-null     int64         
 7   away_goal                          27 non-null     int64         
 8   away_team                          27 non-null     object        
 9   result                             27 non-null     int64         
 10  home_iy_goal                       27 no